In [143]:
import numpy as np
from pathlib import Path
from skimage import io
from skimage import color
from skimage import measure
from skimage import morphology
from skimage import transform
%matplotlib qt

In [285]:
def detect_lines_of_text(img):    
    # szukamy miejsc, w których jasność jest większa i te miejsca zaznaczamy (całe wiersze)
    sum_of_rows = np.sum(img, axis=1) 
    mean_row_value = np.mean(sum_of_rows)
    for i, row in enumerate(img):
        if np.sum(row) < mean_row_value:
            row = np.zeros(row.shape)
            img[i] = row
            
#     img = img.T
#     sum_of_columns = np.sum(img, axis=1)
#     mean_column_value = np.mean(sum_of_columns)
#     for i, row in enumerate(img):
#         if np.sum(row) < mean_column_value:
#             row = np.zeros(row.shape)
#             img[i] = row
    
#     img = img.T
    
    for i, row in enumerate(img):
        if np.mean(row) > 0.0:
            img[i:i+1, :] = 1 
            
    # łączenie lekko rozdzielonych linijek
    img = morphology.dilation(img, morphology.disk(5))
                
    return img
            

def detect_words_in_line(image_org, coords_of_line):
    # wycinamy kawałek obrazu będącego linią tekstu i obracamy go (.T)
    line_img = get_slice_of_image_with_specific_coords(image=image_org, coords=coords_of_line).T
    line_img = morphology.dilation(line_img, morphology.disk(17))
    
    # szukamy miejsc, w których jasność jest większa i te miejsca zaznaczamy w wycinku obrazu
    # (całe wiersze, tutaj poprzez obrócenie są to pojedyczne wyrazy)
    sum_of_rows = np.sum(line_img, axis=1) 
    mean_row_value = np.mean(sum_of_rows)
#     for i, row in enumerate(line_img):
#         if np.sum(row) < mean_row_value:
#             row = np.ones(row.shape)
#             line_img[i] = row
            
#     line_img = line_img.T
#     sum_of_columns = np.sum(line_img, axis=1)
#     mean_column_value = np.mean(sum_of_columns)
#     for i, row in enumerate(line_img):
#         if np.sum(row) < mean_column_value:
#             row = np.zeros(row.shape)
#             line_img[i] = row 
#     line_img = line_img.T
    
#     line_img = color.gray2rgb(line_img)
    # 
    for i, row in enumerate(line_img):
        if np.mean(row) > 0.0:
#             line_img[i:i+1, :] = (255, 0, 0)
            line_img[i:i+1, :] = 255

    line_img = line_img.T

    first_point = coords_of_line[0]
    last_point = coords_of_line[-1]
    image_org[first_point[0]:last_point[0]+1, first_point[1]:last_point[1]+1] = line_img
                
    return image_org


def get_slice_of_image_with_specific_coords(image, coords):
    height = coords[:, 0]
    width = coords[:, 1]
    slice_image = image[(height, width)].reshape((-1, image.shape[1]))
    
    return slice_image

In [286]:
# wykrywanie wyrazów
source_path = Path('../data/partial_results/canny_adam/')
save_path = Path('../data/partial_results/histogram_canny_adam')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.png")
for image_path in images_paths:
    image_path = "../data/partial_results/canny_adam/file1.png"
    print(image_path)
    img = io.imread(image_path)
    img = color.rgb2gray(img)
    img = morphology.closing(img, morphology.disk(1))
    img = morphology.opening(img, morphology.disk(1))
    img_org = img.copy()
    
    img_detected_rows = detect_lines_of_text(img) 

    # szukanie wyrazów w kolejnych linijkach
    label_image = measure.label(img_detected_rows)
    regions = measure.regionprops(label_image)
    print("regions: ", len(regions))

    for region in regions:
        img_org = detect_words_in_line(image_org=img_org, coords_of_line=region.coords)

    io.imshow(img_org)
    break

../data/partial_results/canny_adam/file1.png
regions:  14
